<a href="https://colab.research.google.com/github/GeorgeShishkanov/university_project/blob/main/Token_classificayin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading

In [2]:
!unzip /content/FactRuEval.zip

Archive:  /content/FactRuEval.zip
  inflating: dev.txt                 
  inflating: test.txt                
  inflating: train.txt               


In [ ]:
!pip install transformers
!pip install datasets
!pip install seqeval
!pip install evaluate
!pip install huggingface_hub
!pip install accelerate

In [5]:
import pandas as pd
import re
import csv
from datasets import Dataset
import evaluate
import numpy as np

In [54]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


# Конвертация в csv

In [6]:
with open('/content/train.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('train.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)
with open('/content/dev.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('val.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)
with open('/content/test.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('test.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)

# Обработка датасета

In [90]:
DATA_TRAIN = pd.read_csv('/content/train.csv', sep='\n')
DATA_VAL = pd.read_csv('/content/val.csv', sep='\n')

TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []
count_place = []
count = 0

train_df,val_df  = pd.DataFrame(), pd.DataFrame()

In [91]:
def element_distributor(element):
    element_after_distributor = element.split(' ')
    TOKEN_LIST.append(element_after_distributor[0])
    NER_TAG_LIST.append(element_after_distributor[1])

In [92]:
def function(DATA, df):
  DATA[DATA.columns[0]].apply(element_distributor)
  for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
      tmp1_list=[]
      tmp2_list=[]
      tmp1_list.append(token)
      tmp2_list.append(tag)
      if token == '.':
          list_with_seq.append(tmp1_list)
          list_with_ner.append(tmp2_list)
  df['ner_tags'] = list_with_ner
  df['tokens'] = list_with_seq
  return df 

In [93]:
DATA_TRAIN['title,intro'].apply(element_distributor)
print('ready')

ready


In [94]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]
    

In [95]:
train_df['ner_tags'] = list_with_ner
train_df['tokens'] = list_with_seq

In [96]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [174]:
DATA_TEST = pd.read_csv('/content/test.csv', sep='\n')
test_df = pd.DataFrame()

In [175]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [176]:
DATA_TEST['title,intro'].apply(element_distributor)
print('ready')

ready


In [177]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]

In [178]:
test_df['ner_tags'] = list_with_ner
test_df['tokens'] = list_with_seq

In [173]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [101]:
DATA_VAL['title,intro'].apply(element_distributor)
print('ready')

ready


In [102]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]

In [103]:
val_df['ner_tags'] = list_with_ner
val_df['tokens'] = list_with_seq

In [180]:
test_df

,ner_tags,tokens
0,"[O, O, O, O, O, O, O, O, O, O, O, O, O]","[Тогда, замешанные, в, скандале, прокуроры, от..."
1,"[O, O, O, B-ORG, O, O, B-ORG, I-ORG, B-LOC, B-...","[Символичным, назвал, председатель, РФМ, ,, де..."
2,"[O, O, O, O, O, O, B-PER, I-PER, O, B-PER, I-P...","[На, посту, гендиректора, Yahoo, !, B-ORG, Кэр..."
3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Считаю, невозможным, руководить, областью, с,..."
4,"[B-PER, O, B-PER, I-PER, O, I-PER, O, O, O, O,...","[Боксер, победила, Карли, Фиорину, (, Carly, F..."
...,...,...
2563,"[O, B-LOC, B-PER, I-PER, O, O, O, O, O, O, B-P...","[Президент, Чехии, Милош, Земан, во, вторник, ..."
2564,"[O, O, O, O, O, O, O, O, B-ORG, I-ORG, B-PER, ...","[``, Специальному, городу, —, специальный, реж..."
2565,"[O, B-PER, O, O, O, O, B-PER, I-PER, O, O, B-P...","[Кандидатура, Дубинина, уже, согласована, с, п..."
2566,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[Ему, придется, ``, поближе, познакомиться, с,..."


# Datasets

In [179]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [106]:
from datasets.features import ClassLabel

features = train_dataset.features
column_names = train_dataset.column_names
text_column_name = "tokens"
label_column_name = "ner_tags"

def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(train_dataset[label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)

In [110]:
label_names = list(id2label.values())

In [111]:
def conver_label_to_id(text_labels):
  id_list = []
  for label in text_labels:
    for k,v in label2id.items():
      if  k == label:
        id_list.append(v)
  text_labels = id_list
  return  text_labels

In [112]:
train_df['ner_tags'] = train_df['ner_tags'].apply(conver_label_to_id)

In [113]:
test_df['ner_tags'] = test_df['ner_tags'].apply(conver_label_to_id)

In [114]:
val_df['ner_tags'] = val_df['ner_tags'].apply(conver_label_to_id)

In [115]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

# Model

In [117]:
model_checkpoint = "bert-base-multilingual-cased"

In [147]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForMaskedLM.from_pretrained(model_checkpoint, id2label=id2label, label2id=label2id,)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file bert-base-multilingual-cased/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-LOC",
    "1": "B-ORG",
    "2": "B-PER",
    "3": "I-LOC",
    "4": "I-ORG",
    "5": "I-PER",
    "6": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 0,
    "B-ORG": 1,
    "B-PER": 2,
    "I-LOC": 3,
    "I-ORG": 4,
    "I-PER": 5,
    "O": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,


In [122]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for i in word_ids:
        if i == None:
          new_labels.append(-100)
        else:
          new_labels.append(labels[i])

    return new_labels

In [ ]:
inputs = tokenizer(train_dataset["tokens"][0],is_split_into_words=True)
inputs.tokens()

In [120]:
word_ids = inputs.word_ids()

In [121]:
labels = train_dataset["ner_tags"][0]

In [123]:
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 6, 6, 6, 6, 6, 6, 6, 6, 2, 6]
[-100, 6, 6, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 2, 6, -100]


In [124]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [125]:
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/8 [00:00<?, ?ba/s]

In [126]:
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [127]:
tokenized_val_dataset = val_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [128]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [129]:
batch = data_collator([tokenized_train_dataset[i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    6,    6,    2,    2,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    1,    1,    1,    4,    4,    4,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    2,    2,    2,    6, -100],
        [-100,    6,    6,    6,    1,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    2,    2,    2,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    0,    0,    0,    0,    0,
            6,    6,    6, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, 

In [46]:
metric = evaluate.load("seqeval")

In [132]:
label_names

['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']

In [136]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    model_checkpoint,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [146]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/Grizzlygg/bert-base-multilingual-cased into local empty directory.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7701
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2889
  Number of trainable parameters = 177268231


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.068300,0.054980,0.936738,0.950450,0.943544,0.983869
2,0.021100,0.042807,0.958009,0.973505,0.965695,0.990225
3,0.009800,0.037917,0.970644,0.975262,0.972947,0.991835


***** Running Evaluation *****
  Num examples = 2569
  Batch size = 8
Saving model checkpoint to bert-base-multilingual-cased/checkpoint-963
Configuration saved in bert-base-multilingual-cased/checkpoint-963/config.json
Model weights saved in bert-base-multilingual-cased/checkpoint-963/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased/checkpoint-963/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased/checkpoint-963/special_tokens_map.json
tokenizer config file saved in bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2569
  Batch size = 8
Saving model checkpoint to bert-base-multilingual-cased/checkpoint-1926
Configuration saved in bert-base-multilingual-cased/checkpoint-1926/config.json
Model weights saved in bert-base-multilingual-cased/checkpoint-1926/pytorch_model.bin
tokenizer config file saved i

TrainOutput(global_step=2889, training_loss=0.02841100803334311, metrics={'train_runtime': 615.5837, 'train_samples_per_second': 37.53, 'train_steps_per_second': 4.693, 'total_flos': 886705204684134.0, 'train_loss': 0.02841100803334311, 'epoch': 3.0})

In [148]:
trainer.push_to_hub(commit_message="Training complete")

Saving model checkpoint to bert-base-multilingual-cased
Configuration saved in bert-base-multilingual-cased/config.json
Model weights saved in bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/676M [00:00<?, ?B/s]

Upload file runs/Dec13_19-22-55_3d5a82bb11ce/events.out.tfevents.1670959399.3d5a82bb11ce.71.6:  50%|####9     …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Grizzlygg/bert-base-multilingual-cased
   9116433..e63d363  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Grizzlygg/bert-base-multilingual-cased
   9116433..e63d363  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.9706439393939394}, {'name': 'Recall', 'type': 'recall', 'value': 0.9752616555661275}, {'name': 'F1', 'type': 'f1', 'value': 0.9729473184622687}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9918345872573262}]}
To https://huggingface.co/Grizzlygg/bert-base-multilingual-cased
   e63d363..1b1086c  main -> main

   e63d363..1b1086c  main -> main



'https://huggingface.co/Grizzlygg/bert-base-multilingual-cased/commit/e63d3630c30e3c3bb779136d204d4cd94982282b'

In [149]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_val_dataset, collate_fn=data_collator, batch_size=8
)

In [150]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file bert-base-multilingual-cased/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-LOC",
    "1": "B-ORG",
    "2": "B-PER",
    "3": "I-LOC",
    "4": "I-ORG",
    "5": "I-PER",
    "6": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 0,
    "B-ORG": 1,
    "B-PER": 2,
    "I-LOC": 3,
    "I-ORG": 4,
    "I-PER": 5,
    "O": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_ty

In [151]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
!pip install accelerate

In [154]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [155]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [156]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'Grizzlygg/bert-finetuned-ner-accelerate'

In [157]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/Grizzlygg/bert-finetuned-ner-accelerate into local empty directory.


In [158]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [159]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/2889 [00:00<?, ?it/s]

Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 0: {'precision': 0.9601844397277318, 'recall': 0.9621562156215622, 'f1': 0.9611693164334383, 'accuracy': 0.9892986794338928}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 1: {'precision': 0.9698455683232087, 'recall': 0.9715521665811276, 'f1': 0.97069811735404, 'accuracy': 0.9907823433471878}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Several commits (2) will be pushed upstream.
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 2: {'precision': 0.9695528068506185, 'recall': 0.9685603568033926, 'f1': 0.9690563277249451, 'accuracy': 0.9905718945651601}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Several commits (3) will be pushed upstream.


In [160]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

Configuration saved in bert-finetuned-ner-accelerate/config.json
Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin


In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "Grizzlygg/bert-base-multilingual-cased"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [181]:
test_dataset

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 2568
})

In [210]:
example = test_dataset['tokens'][1]

In [211]:
' '.join(example)

'Символичным назвал председатель РФМ , депутат Государственной думы РФ Леонид Слуцкий ( ЛДПР ) установку памятника Твардовскому в день годовщины Великой Отечественной войны .'

In [212]:
token_classifier('Символичным назвал председатель РФМ , депутат Государственной думы РФ Леонид Слуцкий ( ЛДПР ) установку памятника Твардовскому в день годовщины Великой Отечественной войны')

[{'entity_group': 'ORG',
  'score': 0.99896204,
  'word': 'РФ',
  'start': 32,
  'end': 34},
 {'entity_group': 'ORG',
  'score': 0.999097,
  'word': '##М',
  'start': 34,
  'end': 35},
 {'entity_group': 'ORG',
  'score': 0.99904156,
  'word': 'Государственной думы',
  'start': 46,
  'end': 66},
 {'entity_group': 'LOC',
  'score': 0.99839467,
  'word': 'РФ',
  'start': 67,
  'end': 69},
 {'entity_group': 'PER',
  'score': 0.99982786,
  'word': 'Леонид Слуцкий',
  'start': 70,
  'end': 84},
 {'entity_group': 'ORG',
  'score': 0.998528,
  'word': 'Л',
  'start': 87,
  'end': 88},
 {'entity_group': 'ORG',
  'score': 0.9984641,
  'word': '##Д',
  'start': 88,
  'end': 89},
 {'entity_group': 'ORG',
  'score': 0.9983399,
  'word': '##П',
  'start': 89,
  'end': 90},
 {'entity_group': 'ORG',
  'score': 0.9983551,
  'word': '##Р',
  'start': 90,
  'end': 91},
 {'entity_group': 'PER',
  'score': 0.9954716,
  'word': 'Т',
  'start': 114,
  'end': 115},
 {'entity_group': 'PER',
  'score': 0.996586